In [51]:
import numpy as np
import pandas as pd
pd.set_option('max_colwidth', None)
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import SpatialDropout1D, Dense, Embedding, SimpleRNN, LSTM, Bidirectional, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from nltk import word_tokenize


In [11]:
df=pd.read_csv('Movieupdate.csv')

In [12]:
df

,text,label
0,grew b watching loving thunderbird mate school watched played thunderbird school lunch school wanted virgil scott one wanted alan counting became art form took child see movie hoping would get glimpse loved child bitterly disappointing high point snappy theme tune could compare original score thunderbird thankfully early saturday morning one television channel still play rerun series gerry anderson wife created jonatha frakes hand director chair version completely hopeless waste film utter rubbish cgi remake may acceptable replacing marionette homo sapiens subsp sapiens huge error judgment,0
1,put movie dvd player sat coke chip expectation hoping movie would contain strong point first movie awsome animation good flowing story excellent voice cast funny comedy kick as soundtrack disappointment found atlantis milo return read review first might let following paragraph directed seen first movie enjoyed primarily point mentioned first scene appears shock picked atlantis milo return display case local videoshop whatever expectation music feel bad imitation first movie voice cast replaced fitting one exception character like voice sweet actual drawing bad animation particular sad sight storyline also pretty weak like three episode schooby doo single adventurous story got last time misunderstand good schooby doo episode laugh single time although might sniggered twice audience seen first movie especially care similar sequel fast review movie stand alone product liked schooby doo might like movie could still enjoy movie nothing else suspect might good kid movie would know might better milo return three episode series cartoon channel breakfast tv,0
2,people know particular time past like feel need try define time others replace woodstock civil war apollo moon landing titanic sinking got realistic flick formulaic soap opera populated entirely low life trash kid young allowed go woodstock failed grade school composition show old meany put movie prove know nuttin topic still make money yeah already know one thing watching film give little insight underclass thinking next time see slut bar look like diane lane running way child abuse let parent worthless raise kid audience abuse simply stick woodstock moonlanding flick ipso facto mean film portrays,0
3,even though great interest biblical movie bored death every minute movie everything bad movie long acting time joke script horrible get point mixing story abraham noah together value time sanity stay away horror,0
4,die hard dad army fan nothing ever change got tape dvd audiobooks every time watch listen brand new film film run certain episode man hour enemy within gate battle school numerous others different edge introduction new general instead captain square brilliant move especially would cash cheque something rarely done follows early year getting equipment uniform starting training great film boring sunday afternoon two draw back one german bogus dodgy accent come one german pronounced letter w like u two casting liz frazer instead familiar janet davis like liz film like carry ons carry correctly janet davis would better choice,1
...,...,...
39718,western union something forgotten classic western perhaps reason lie fact unavailability dvd united state however lost appeared region england blessing way incongruous totally ironic one considers movie depicting founding establishment uniquely american organization western union telegraph company without region release beggar belief simply make sense produced fox western union directed fritz lang second occasion great german director undertook direct western done excellent job year fox return frank james would one western outing splendid rancho notorious lang ford hawk western union turned fine solid western hold well beautifully photographed early three strip technicolor edward cronjager boasted good cast headed robert young randolph scott dean jagger female lead taken virginia gilmore really little picture actress never made 

In [22]:
def CountSentence(row):
    token=word_tokenize(row)
    length=len(token)
    return length

In [23]:
df['sentenceLength']= df['text'].apply(lambda x: CountSentence(x))

In [24]:
df['sentenceLength'].describe()

count    39723.000000
mean       118.400826
std         89.495887
min          3.000000
25%         63.000000
50%         88.000000
75%        144.000000
max       1417.000000
Name: sentenceLength, dtype: float64

In [36]:
df[df['sentenceLength']>300].count()

text              2084
label             2084
sentenceLength    2084
dtype: int64

##### Tokenize and convert to vector

In [13]:
tokenizer=Tokenizer(num_words=2000)

In [14]:
tokenizer.fit_on_texts(df['text'].values)

In [15]:
x= tokenizer.texts_to_sequences(df['text'].values)

##### Pad Sequence

In [37]:
X=pad_sequences(x, maxlen=300)

In [38]:
X

array([[   0,    0,    0, ...,  817,  108,  507],
       [   0,    0,    0, ...,  630,  880,  134],
       [   0,    0,    0, ...,  277,  171,    2],
       ...,
       [   0,    0,    0, ...,    5, 1158,  463],
       [   0,    0,    0, ...,  111,  634,  486],
       [   0,    0,    0, ...,  918,  222,  342]])

In [39]:
y=df['label']

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)

In [42]:
X_train.shape

(31778, 300)

In [43]:
from tensorflow.keras.utils import to_categorical
y_train_cat=to_categorical(y_train)
y_test_cat=to_categorical(y_test)

##### RNN Model

In [44]:
model = Sequential()
model.add(Embedding(2000, 100, input_length=X.shape[1]))
model.add(SimpleRNN(100))
model.add(Dense(2, activation='sigmoid'))

#Compile the model
model.compile(optimizer='adam', loss= 'binary_crossentropy', metrics=['accuracy'])

In [46]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 100)          200000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 100)               20100     
                                                                 
 dense (Dense)               (None, 2)                 202       
                                                                 
Total params: 220,302
Trainable params: 220,302
Non-trainable params: 0
_________________________________________________________________


In [47]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [48]:
es=EarlyStopping(patience=3, verbose=1)
chkpt=ModelCheckpoint('./Models/rnn.hdf5', verbose=1, save_best_only=True)

In [50]:
hist=model.fit(X_train, y_train_cat, batch_size=32, epochs=10, validation_split=0.2, callbacks=[es,chkpt])

Epoch 1/10
795/795 [==============================] - ETA: 0s - loss: 0.5278 - accuracy: 0.7230
Epoch 1: val_loss improved from inf to 0.39428, saving model to ./NLP project/Models\rnn.hdf5
795/795 [==============================] - 261s 325ms/step - loss: 0.5278 - accuracy: 0.7230 - val_loss: 0.3943 - val_accuracy: 0.8206
Epoch 2/10
795/795 [==============================] - ETA: 0s - loss: 0.3701 - accuracy: 0.8438
Epoch 2: val_loss improved from 0.39428 to 0.37455, saving model to ./NLP project/Models\rnn.hdf5
795/795 [==============================] - 254s 319ms/step - loss: 0.3701 - accuracy: 0.8438 - val_loss: 0.3745 - val_accuracy: 0.8301
Epoch 3/10
795/795 [==============================] - ETA: 0s - loss: 0.4122 - accuracy: 0.8166
Epoch 3: val_loss did not improve from 0.37455
795/795 [==============================] - 257s 324ms/step - loss: 0.4122 - accuracy: 0.8166 - val_loss: 0.5770 - val_accuracy: 0.6981
Epoch 4/10
795/795 [==============================] - ETA: 0s - loss

##### Model Improvement

In [55]:
model3 = Sequential()
model3.add(Embedding(2000, 100, input_length=X.shape[1]))
model3.add(Dropout(0.3))
model3.add(SimpleRNN(100))
model3.add(Dense(2, activation='sigmoid'))

#Compile the model
model3.compile(optimizer='adam', loss= 'binary_crossentropy', metrics=['accuracy'])

model3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 300, 100)          200000    
                                                                 
 dropout_1 (Dropout)         (None, 300, 100)          0         
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 100)               20100     
                                                                 
 dense_2 (Dense)             (None, 2)                 202       
                                                                 
Total params: 220,302
Trainable params: 220,302
Non-trainable params: 0
_________________________________________________________________


In [56]:
es3=EarlyStopping(patience=3, verbose=1)
chkpt3=ModelCheckpoint('./Models/rnn2.hdf5', verbose=1, save_best_only=True)

In [58]:
hist3=model3.fit(X_train, y_train_cat, batch_size=32, epochs=10, validation_split=0.2, callbacks=[es3,chkpt3])

Epoch 1/10
795/795 [==============================] - ETA: 0s - loss: 0.6464 - accuracy: 0.6162
Epoch 1: val_loss improved from inf to 0.65788, saving model to ./Models\rnn2.hdf5
795/795 [==============================] - 264s 331ms/step - loss: 0.6464 - accuracy: 0.6162 - val_loss: 0.6579 - val_accuracy: 0.6156
Epoch 2/10
795/795 [==============================] - ETA: 0s - loss: 0.5586 - accuracy: 0.7159
Epoch 2: val_loss improved from 0.65788 to 0.45809, saving model to ./Models\rnn2.hdf5
795/795 [==============================] - 259s 326ms/step - loss: 0.5586 - accuracy: 0.7159 - val_loss: 0.4581 - val_accuracy: 0.8016
Epoch 3/10
795/795 [==============================] - ETA: 0s - loss: 0.4638 - accuracy: 0.7834
Epoch 3: val_loss did not improve from 0.45809
795/795 [==============================] - 257s 323ms/step - loss: 0.4638 - accuracy: 0.7834 - val_loss: 0.5998 - val_accuracy: 0.6751
Epoch 4/10
795/795 [==============================] - ETA: 0s - loss: 0.4241 - accuracy: 0

##### LSTM

In [52]:
model2 = Sequential()
model2.add(Embedding(2000, 100, input_length=X.shape[1]))
model2.add(SpatialDropout1D(0.3))
model2.add(LSTM(100))
model2.add(Dropout(0.5))
model2.add(Dense(2, activation='sigmoid'))

#Compile the model
model2.compile(optimizer='adam', loss= 'binary_crossentropy', metrics=['accuracy'])

model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 300, 100)          200000    
                                                                 
 spatial_dropout1d (SpatialD  (None, 300, 100)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 202       
                                                                 
Total params: 280,602
Trainable params: 280,602
Non-trainable params: 0
________________________________________________

In [53]:
es2=EarlyStopping(patience=3, verbose=1)
chkpt2=ModelCheckpoint('./Models/lstm.hdf5', verbose=1, save_best_only=True)

In [54]:
hist2=model2.fit(X_train, y_train_cat, batch_size=32, epochs=5, validation_split=0.2, callbacks=[es2,chkpt2])

Epoch 1/5
795/795 [==============================] - ETA: 0s - loss: 0.3946 - accuracy: 0.8224
Epoch 1: val_loss improved from inf to 0.32185, saving model to ./Models\lstm.hdf5
795/795 [==============================] - 30s 32ms/step - loss: 0.3946 - accuracy: 0.8224 - val_loss: 0.3219 - val_accuracy: 0.8617
Epoch 2/5
795/795 [==============================] - ETA: 0s - loss: 0.3065 - accuracy: 0.8759
Epoch 2: val_loss improved from 0.32185 to 0.31138, saving model to ./Models\lstm.hdf5
795/795 [==============================] - 25s 32ms/step - loss: 0.3065 - accuracy: 0.8759 - val_loss: 0.3114 - val_accuracy: 0.8711
Epoch 3/5
795/795 [==============================] - ETA: 0s - loss: 0.3182 - accuracy: 0.8712
Epoch 3: val_loss did not improve from 0.31138
795/795 [==============================] - 25s 31ms/step - loss: 0.3182 - accuracy: 0.8712 - val_loss: 0.3368 - val_accuracy: 0.8716
Epoch 4/5
793/795 [============================>.] - ETA: 0s - loss: 0.2639 - accuracy: 0.8954
Epoc